# Readme

This runs after the creation of the pivoted actors part-00000-6b4a5812-38da-4d40-bd52-59c8c6d0ddb0-c000.csv, the master data part-00000-b7bb5828-f78a-4d39-a019-33f1586faf35-c000.csv

Create a new dictionary to support Chroma DB

    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]

Where DB == imdb_title_titleId

# WARNING - HAD TO DROP RECORD tt12853970

# Import data

In [1]:
import pandas as pd
import json

In [2]:
master_df = pd.read_csv("data/export/imdb_imdbscrape_kaggle_master.csv", delimiter="|")
actors_df = pd.read_csv('data/export/imdb_principals_name_basics_pivot.csv', delimiter="|")


In [3]:
master_df.head(10)

# get master_df colum nnames
master_df.columns

# check where anime_other_name is Ace Attorney
master_df[master_df['anime_English name'] == 'Ace Attorney']

,imdb_scrape_imdb_id,imdb_title_attributes,imdb_title_isOriginalTitle,imdb_title_language,imdb_title_ordering,imdb_title_region,imdb_title_title,imdb_title_titleId,imdb_title_types,anime_Aired,...,anime_Rank,anime_Rating,anime_Score,anime_Scored By,anime_Source,anime_Status,anime_Studios,anime_Synopsis,anime_Type,anime_anime_id
2827,tt5603356,\\N,0,\\N,12,US,Ace Attorney,tt5603356,imdbDisplay,NaN,...,NaN,NaN,6.5,NaN,NaN,NaN,NaN,"Since he was a child, Ryuuichi Naruhodou's dre...",NaN,31630.0


In [4]:
actors_df.head(10)


,imdb_principals_tconst,imdb_name_basics_primaryName
0,tt0051694,"Alvin Schoncite, Shin Uehara, Seiichi Yashiro,..."
1,tt0053275,"Teruo Miyazaki, Mitsuaki Ishikawa, Akira Daiku..."
2,tt0054265,"Daisaku Shirakawa, Goro Kontaibo, Hideo Kinosh..."
3,tt0054631,"Jô Mizuki, Isuzu Yamada, Hajime Kaburagi, Hiro..."
4,tt0057669,"Yukiko Okada, Mitsuaki Ishikawa, Ichirô Ikeda,..."
5,tt0058339,"Adolf Hitler, Shinsuke Chikaishi, Osamu Tezuka"
6,tt0058342,Osamu Tezuka
7,tt0058734,"Daisaku Shirakawa, Hideo Kinoshita, Osamu Tezu..."
8,tt0058807,"Sonia Owens, Peter Fernandez, Ray Owens, Billi..."
9,tt0059212,"Chiyoko Honma, Hiroshi Ohkawa, Kyû Sakamoto, Y..."


# Clean Data

In [5]:
# Join master_df and actors_df, where imdb_title_titleId = imdb_principals_tconst

master_actorpivot = master_df.merge(actors_df, left_on='imdb_title_titleId', right_on='imdb_principals_tconst')


In [6]:
master_actorpivot.head()

,imdb_scrape_imdb_id,imdb_title_attributes,imdb_title_isOriginalTitle,imdb_title_language,imdb_title_ordering,imdb_title_region,imdb_title_title,imdb_title_titleId,imdb_title_types,anime_Aired,...,anime_Score,anime_Scored By,anime_Source,anime_Status,anime_Studios,anime_Synopsis,anime_Type,anime_anime_id,imdb_principals_tconst,imdb_name_basics_primaryName
0,tt21030032,\\N,0,ja,16,JP,【推しの子】,tt21030032,imdbDisplay,NaN,...,8.98,NaN,NaN,NaN,NaN,"In the entertainment world, celebrities often ...",NaN,52034.0,tt21030032,"Natalie Rial, Christina Marie Kelly, Takeo Ots..."
1,tt1343075,\\N,0,\\N,1,JP,.hack//G.U. Returner,tt1343075,\\N,NaN,...,6.67,NaN,NaN,NaN,NaN,The characters from previous .hack//G.U. Games...,NaN,2928.0,tt1343075,"Kaori Nazuka, Ayako Kawasumi, Takahiro Sakurai..."
2,tt1164545,\\N,1,\\N,2,\\N,.hack//G.U. Trilogy,tt1164545,original,NaN,...,6.36,NaN,NaN,NaN,NaN,A special bonus Parody Mode added to the extra...,NaN,4469.0,tt1164545,"Hiroshi Matsuyama, Ayako Kawasumi, Saeko Hirot..."
3,tt0823406,\\N,0,\\N,1,US,.hack//GIFT,tt0823406,imdbDisplay,"Nov 16, 2003",...,6.1,9694.0,Original,Finished Airing,Bee Train,As an expression of gratitude for the heroes o...,OVA,454.0,tt0823406,"Isshin Chiba, Shin'ya Kawatsura, Sayaka Aida, ..."
4,tt0361140,\\N,0,\\N,4,US,.hack//SIGN,tt0361140,imdbDisplay,NaN,...,6.95,NaN,NaN,NaN,NaN,"A young wavemaster, only known by the alias of...",NaN,48.0,tt0361140,"Amanda Winn Lee, Kaori Nazuka, Donna Rawlins, ..."


In [7]:
# Create a new dictionary to support chroma DB
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# where
# The ids field is imdb_title_titleId
# And each column is a new key-value pair under metadata

# Create a basic list
# For each row
#     Create a new dictionary
#     Add the key-value pairs, where the column name is the key and the value is the value
#     Add the new dictionary to the metadatas list

metadata = []
for index, row in master_actorpivot.iterrows():
    metadata_row = {}
    for column in master_actorpivot.columns:
        metadata_row[column] = row[column]
    metadata.append(metadata_row)
    # print(metadata)

# export the metadata object to a json file
import json
with open('data/export/metadata.json', 'w') as outfile:
    json.dump(metadata, outfile)
    

    

In [9]:
ids_imdb = list(master_actorpivot['imdb_title_titleId'])
with open('data/export/ids_imdb_titleID.json', 'w') as outfile:
    json.dump(ids_imdb, outfile)

ids_anime_anime_id = list(master_actorpivot['anime_anime_id'])
with open('data/export/ids_anime_anime_id.json', 'w') as outfile:
    json.dump(ids_anime_anime_id, outfile)